In [ ]:
import music21
import pathlib

from thesession.converter import ABCMusicConverter

In [ ]:
ABCMusicConverter("cooleys.abc", "cooleys").to_mp3(
    instrument="flute",
    tempo=190,
    max_notes=300,
    cut_silence=30,
    start=0.57,
    noise_amplitude=0.001,
    duration=120,
    vbr=8,
    clean_files=True
)

# CLAP

In [ ]:
import torch
from laion_clap import CLAP_Module

# Load pretrained CLAP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLAP_Module(enable_fusion=True) # For using fusion to manage >10s clips
model.load_ckpt()  # downloads pretrained weights
model = model.to(device)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maxime/.venvs/session/lib64/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

In [11]:
import torchaudio

waveform, sr = torchaudio.load("cooleys.mp3")

In [12]:
import torchaudio
import torch


def load_audio(filepath, target_sr=16000):
    waveform, sr = torchaudio.load(filepath)
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, sr, target_sr)
    return waveform.mean(dim=0).unsqueeze(0)  # Convert to mono, add batch dim

In [13]:
audio_tensor0 = load_audio("audio/1_cooleys/36371_0.mp3")
audio_tensor1 = load_audio("audio/1_cooleys/36371_1.mp3")
audio_tensor2 = load_audio("audio/8_the_banshee/32846_0.mp3")
audio_tensor3 = load_audio("audio/8_the_banshee/32846_1.mp3")

In [14]:
with torch.no_grad():
    embedding0 = model.get_audio_embedding_from_data(audio_tensor0, use_tensor=True)
    embedding1 = model.get_audio_embedding_from_data(audio_tensor1, use_tensor=True)
    embedding2 = model.get_audio_embedding_from_data(audio_tensor2, use_tensor=True)
    embedding3 = model.get_audio_embedding_from_data(audio_tensor3, use_tensor=True)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


In [15]:
embeddings = torch.cat([embedding0, embedding1, embedding2, embedding3], dim=0)
embeddings.shape

torch.Size([4, 512])

In [16]:
import torch.nn.functional as F

F.cosine_similarity(embeddings[None, :, :], embeddings[:, None, :], dim=-1)

tensor([[1.0000, 0.8502, 0.8558, 0.8473],
        [0.8502, 1.0000, 0.9414, 0.8412],
        [0.8558, 0.9414, 1.0000, 0.8524],
        [0.8473, 0.8412, 0.8524, 1.0000]], device='cuda:0')

In [14]:
import torch
import torch.nn.functional as F


def nt_xent_loss(z1, z2, temperature=0.07):
    """
    Contrastive loss using implicit negatives (NT-Xent).
    Args:
        z1: Tensor of shape (N, D) – embeddings from view 1 (e.g., anchors)
        z2: Tensor of shape (N, D) – embeddings from view 2 (e.g., positives)
    Returns:
        Scalar contrastive loss
    """
    batch_size = z1.size(0)

    # Concatenate for full 2N x D
    z = torch.cat([z1, z2], dim=0)  # shape: (2N, D)

    # Cosine similarity (2N x 2N)
    sim = F.cosine_similarity(z[None, :, :], z[:, None, :], dim=-1)

    # Mask self-similarity
    mask = torch.eye(2 * batch_size, device=z.device).bool()
    sim.masked_fill_(mask, -float("inf"))  # ignore similarity to self

    # Targets: for i in 0..N-1, positive pair is i<->i+N and i+N<->i
    targets = torch.cat(
        [torch.arange(batch_size, 2 * batch_size), torch.arange(0, batch_size)]
    ).to(z.device)

    loss = F.cross_entropy(sim / temperature, targets)
    return loss

In [9]:
loss =nt_xent_loss(embeddings[0:2], embeddings[2:])
loss

tensor(2.3978, device='cuda:0')

# Dataset

In [11]:
import pathlib
import numpy as np

from thesession.dataset import TheSessionDataset

path = pathlib.Path("audio_flac")
tunes = [p.stem for p in path.iterdir() if p.is_dir()]
tunes.sort()

prng = np.random.default_rng(42)
tunes = prng.permutation(tunes)
tunes

array(['1970_mhari_mcleans', '9041_tuolpagorni', '19046_soup_of_the_day',
       '19052_the_ruby', '13741_ncken_valsen', '19045_rufty_tufty',
       '1969_joe_murphys_return_from_chicago', '5220_raffertys_mustache',
       '9048_kitty_sean_cunninghams', '5210_bouffe_doigts',
       '5214_reel_de_joliette', '13739_the_free_mason',
       '9046_mild_mabel_kelly', '1966_the_quilty',
       '1971_the_forest_grove', '1973_sorefingers_rant',
       '2_the_bucks_of_oranmore', '19048_logie_bridge',
       '9038_boules_et_guirlandes', '5216_the_templehouse',
       '5213_forest_rogers', '9040_the_hounds_of_letterfrack',
       '5218_kellys', '13738_magnie_o_da_daeks',
       '1967_farewell_to_the_troubles_of_the_world',
       '19043_the_whirlpool', '1974_i_have_no_money',
       '19051_stuck_in_a_good_place', '5208_sandys_new_chanter',
       '13743_the_cross_of_inverness', '9045_the_slippy_wet',
       '19049_the_grave_of_the_unknown_clansman',
       '19044_reel_des_accordeonistes', '13737_t

In [13]:
test_subset = tunes[0 : int(0.15 * len(tunes))]
val_subset = tunes[int(0.15 * len(tunes)) : int(0.3 * len(tunes))]
train_subset = tunes[int(0.3 * len(tunes)):]


#train_dataset = TheSessionDataset("audio_flac", subset=train_subset, sampling_rate=48000, device="cuda")
train_dataset = TheSessionDataset("audio_flac", subset=["1_cooleys"], sampling_rate=48000, device="cuda", fmt=".flac")
len(train_dataset.records)

125

In [15]:
train_dataset[32]

(tensor([-2.1248e-02, -2.6456e-02, -2.8922e-02,  ..., -3.0179e-04,
         -1.5179e-04, -4.8826e-05], device='cuda:0'),
 tensor([-0.0234, -0.0276, -0.0282,  ...,  0.0165,  0.0115,  0.0052],
        device='cuda:0'))

In [16]:
%%timeit

train_dataset[0]

25.4 ms ± 2.26 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
import torch

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

In [18]:
%%timeit

z1, z2 = next(iter(train_loader))

418 ms ± 27.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
z1, z2 = next(iter(train_loader))

In [22]:
z1.shape

torch.Size([16, 2880000])

In [23]:
z1_embed = model.get_audio_embedding_from_data(z1, use_tensor=True)

NameError: name 'model' is not defined

In [12]:
z2_embed = model.get_audio_embedding_from_data(z2, use_tensor=True)

In [16]:
loss = nt_xent_loss(z1_embed, z2_embed)
loss

tensor(3.7870, device='cuda:0', grad_fn=<NllLossBackward0>)

In [17]:
loss.backward()

# Model

In [1]:
from thesession.model import TheSessionModel

model = TheSessionModel()
model.load_ckpt()

/home/maxime/.venvs/session/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maxime/.venvs/session/lib64/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

In [2]:
model.save("models/test.pt")

In [3]:
model2 = TheSessionModel()
model2.load("models/test.pt")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [70]:
model = TheSessionModel(enable_fusion=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [73]:
model.toggle_parameter_gradients(False)

Disabling gradient for parameter clap_model.model.logit_scale_a
Disabling gradient for parameter clap_model.model.logit_scale_t
Disabling gradient for parameter clap_model.model.audio_branch.spectrogram_extractor.stft.conv_real.weight
Disabling gradient for parameter clap_model.model.audio_branch.spectrogram_extractor.stft.conv_imag.weight
Disabling gradient for parameter clap_model.model.audio_branch.logmel_extractor.melW
Disabling gradient for parameter clap_model.model.audio_branch.bn0.weight
Disabling gradient for parameter clap_model.model.audio_branch.bn0.bias
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.proj.weight
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.proj.bias
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.norm.weight
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.norm.bias
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.mel_conv

In [74]:
model.toggle_parameter_gradients(True, ['clap_model.model.audio_projection'])

Enabling gradient for parameter clap_model.model.audio_projection.0.weight
Enabling gradient for parameter clap_model.model.audio_projection.0.bias
Enabling gradient for parameter clap_model.model.audio_projection.2.weight
Enabling gradient for parameter clap_model.model.audio_projection.2.bias


# Entraînement

In [ ]:
import time

# Training function
def train_model(
    model,
    train_dataset,
    val_dataset,
    criterion,
    optimizer,
    epochs=10,
    device=None,
    verbose=True,
    batch_size=32,
    num_workers=0,
    best_model_path="models/best_model.pt"
):
    """Train a pytorch model

    Parameters
    ----------
    model : torch.nn.Module
        The model to be trained
    train_dataset : torch.utils.data.Dataset
        The dataset from which the model is trained
    val_dataset : torch.utils.data.Dataset
        The dataset that is evaluated at each epoch to follow the training process
    criterion : torch.nn.Module
        The module that computes the training loss (e.g. CrossEntropyLoss)
    optimizer : torch.optim.Optimizer
        The optimizer that is used for model training
    epochs : int, default: 10
        The number of epochs that are used to train the model
    device : torch.device, default: None
        The device onto which the model is trained
    verbose : bool, default: True
        Whether a summary of the training process is printed at each epoch or not.
    batch_size: int, default: 32
        The number of data points to load with each batch.
    best_model_path: str, default: "models/best_model.pt"
        The path to save the best model.

    Returns
    -------
    pandas.DataFrame
        A dataframe containing the average training and validation loss at each epoch,
        and the model accuracy on the training and validation data.

    """
    # Path to save the best model
    best_model_path = pathlib.Path(best_model_path)
    best_model_path.parent.mkdir(exist_ok=True)

    # Definition of data loaders
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    )

    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    )

    train_loss_array = np.zeros(epochs)
    val_loss_array = np.zeros(epochs)
    train_accuracy_array = np.zeros((epochs, 3))
    val_accuracy_array = np.zeros((epochs, 3))

    # Start time counter
    start_time = time.perf_counter()

    # Iteration over the epochs
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0.0
        correct = np.zeros(3)

        for i, (X_batch, y_batch) in enumerate(train_loader):
            if verbose:
                print(f"Train batch {i+1}/{len(train_loader)}", end="\r")

            # Send to device
            if device is not None:
                X_batch = X_batch.to(device)
                y_batch = [lbl.to(device) for lbl in y_batch]

            optimizer.zero_grad()

            # Compute prediction
            y_pred = model(X_batch)

            # Compute loss
            loss = 0.0
            for pred, lbl in zip(y_pred, y_batch):
                loss += criterion(pred, lbl)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            # Compute number of correct predictions
            predicted = torch.stack([torch.argmax(pred, axis=1) for pred in y_pred], 1)
            expected = torch.stack([torch.argmax(lbl, axis=1) for lbl in y_batch], 1)
            correct += np.asarray((predicted == expected).sum(axis=0).cpu())

        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = correct / len(train_dataset)

        train_loss_array[epoch] = avg_train_loss
        train_accuracy_array[epoch] = train_accuracy

        # Validation
        model.eval()

        val_loss = 0.0
        correct = np.zeros(3)

        with torch.no_grad():
            for i, (X_batch, y_batch) in enumerate(val_loader):
                if verbose:
                    print(f"Validation batch {i+1}/{len(val_loader)}", end="\r")

                if device is not None:
                    X_batch = X_batch.to(device)
                    y_batch = [lbl.to(device) for lbl in y_batch]

                y_pred = model(X_batch)

                # Compute loss
                loss = 0.0
                for pred, lbl in zip(y_pred, y_batch):
                    loss += criterion(pred, lbl)

                val_loss += loss.item()

                predicted = torch.stack([torch.argmax(pred, axis=1) for pred in y_pred], 1)
                expected = torch.stack([torch.argmax(lbl, axis=1) for lbl in y_batch], 1)
                correct += np.asarray((predicted == expected).sum(axis=0).cpu())

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = correct / len(val_dataset)

        val_loss_array[epoch] = avg_val_loss
        val_accuracy_array[epoch] = val_accuracy

        if avg_val_loss == np.min(val_loss_array[:epoch+1]):
            torch.save(model.state_dict(), best_model_path)

        # Printing a summary
        if verbose:
            print(
                f"Epoch [{epoch+1}/{epochs}] | "
                f"Train Loss: {avg_train_loss:.4f} | "
                f"Train Accuracy: {train_accuracy[0]:.4f}, {train_accuracy[1]:.4f}, {train_accuracy[2]:.4f} | "
                f"Valid. Loss: {avg_val_loss:.4f} | "
                f"Valid. Accuracy: {val_accuracy[0]:.4f}, {val_accuracy[1]:.4f}, {val_accuracy[2]:.4f} | "
            )

    # End time counter
    end_time = time.perf_counter()
    duration = end_time - start_time

    if verbose:
        print(f"Training time for {epochs} epochs: {duration:0.2f} seconds")

    # Regroup results into dataframe
    res = pd.DataFrame(
        {
            "train_loss": train_loss_array,
            "val_loss": val_loss_array,
            "train_accuracy_surface": train_accuracy_array[:,0],
            "train_accuracy_visibility": train_accuracy_array[:,1],
            "train_accuracy_snowblades": train_accuracy_array[:,2],
            "val_accuracy_surface": val_accuracy_array[:,0],
            "val_accuracy_visibility": val_accuracy_array[:,1],
            "val_accuracy_snowblades": val_accuracy_array[:,2],
        }
    )

    return res